In [79]:
import jsonlines
import json
import re
import nltk
import unicodedata
!pip3 install pytextrank
import pytextrank

!pip3 install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

!pip3 install spacy
import spacy

import pandas as pd

!pip3 install scrubadub
import scrubadub
scrubber = scrubadub.Scrubber()

import spacy
nlp = spacy.load('ru_core_news_sm')
nlp.add_pipe("textrank")

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


## Любовь

In [80]:
love_data = []
with jsonlines.open('data.jsonl') as reader:
    for obj in reader:
        love_data.append(obj)
        

In [81]:
len(love_data)

82628

In [151]:
love_data[0]

{'question': 'С мужчиной не видимся',
 'comment': 'Поснакомились с мужчиной месяц назад. Интересно познакомились, сходили в ресторан. На втором свидании был секс. После он писал 2 дня подряд и опять встретились ужинали хорошо провели время и опять был секс. Сейчас часто пишет сам, долго переписываемся он пишет про дела как идут на работе, переживания личные в семье какие-то свои. Чуть что встретимся расскажу, но не виделись с ним почти как уже 2 недели. (((( Почему так?',
 'sub_category': 'relations',
 'author': 'Алиса Томилина',
 'author_rating': {'category': 'Ученик', 'value': '98'},
 'answers': [{'text': 'возможно вы достались ему слишком просто и он для себя решил, что без опасений можно переключится на другие интересы зная о том, что вы никуда не денетесь. Становитесь менее предсказуемой и более загадочной и мужчина не будет так надолго пропадать.',
   'author_rating': {'category': 'Искусственный Интеллект',
    'value': '114621'}},
  {'text': 'Потому что он женат, и постоянные от

In [82]:
def get_love_answers(l):
    answers = []
    for d in l:
        answers.append(d['text'])
    return ' '.join(answers)

In [83]:
# Сначала думали использовать комментарий в качестве контекста,
# но дальше отказались от этой идеии, так как в нем нет намека на ответ.
train_love = love_data[24790:]
test_love = love_data[:24790]

df_love = pd.DataFrame.from_dict(train_love)
df_love = df_love.drop(['sub_category', 'author', 'author_rating','poll'], axis= 1)
df_love['answers'] = df_love['answers'].apply(get_love_answers)
df_love

,question,comment,answers
0,Девушки вопрос немного эротичный... смотрите л...,,Она меня бьёт если не включаю Нет Нет)) Нет- с...
1,"Вы хоть не верите в сказку, что каждого ждет г...",Только в России большинство живущих досих пор ...,"верю.. Верю, но сдается мне живет он, где то н..."
2,Я видел на остановке девушка целует другую дев...,,"Куда? главное, шо не меня, так что да. это лес..."
3,девчат вы хоть понимаете на что подписываетесь?,,на Прочие взаимоотношения расскажи ) ❤❤❤❤❤❤❤❤❤...
4,бывает ли дружба между маленькой 15 летней дев...,,"Дружбы не существует, есть лишь взаимовыгодные..."
...,...,...,...
57833,что написать на листочке девушке в школе? чтоб...,,все что хочешь) только не указывай инициалы и ...
57834,Вы видели Афродиту?))) скульптуру.. ну как кра...,,скульптура мужик какой то Если сказали те тп=ч...
57835,"Как назвать человека, который помогает, даде е...","Не согу вспомнить слово, вот ситуация, тебе ну...",Отзывчивый альтруист Самый верный друг Друг Ал...
57836,Какой подарок подарить девушке на 22-летие?,,"Себя Стриптизёра. Парня. Незабываемый вечер, н..."


In [84]:
# Более осознанным нам показлось сделать те самые обьемные вопросы с форума - контекстом,
# так как там точно есть ответ

df_love = df_love.rename(columns = {'answers':'context', 'comment': 'answer'})
cols = ['context', 'question', 'answer']
df_love = df_love[cols]
df_love

,context,question,answer
0,Она меня бьёт если не включаю Нет Нет)) Нет- с...,Девушки вопрос немного эротичный... смотрите л...,
1,"верю.. Верю, но сдается мне живет он, где то н...","Вы хоть не верите в сказку, что каждого ждет г...",Только в России большинство живущих досих пор ...
2,"Куда? главное, шо не меня, так что да. это лес...",Я видел на остановке девушка целует другую дев...,
3,на Прочие взаимоотношения расскажи ) ❤❤❤❤❤❤❤❤❤...,девчат вы хоть понимаете на что подписываетесь?,
4,"Дружбы не существует, есть лишь взаимовыгодные...",бывает ли дружба между маленькой 15 летней дев...,
...,...,...,...
57833,все что хочешь) только не указывай инициалы и ...,что написать на листочке девушке в школе? чтоб...,
57834,скульптура мужик какой то Если сказали те тп=ч...,Вы видели Афродиту?))) скульптуру.. ну как кра...,
57835,Отзывчивый альтруист Самый верный друг Друг Ал...,"Как назвать человека, который помогает, даде е...","Не согу вспомнить слово, вот ситуация, тебе ну..."
57836,"Себя Стриптизёра. Парня. Незабываемый вечер, н...",Какой подарок подарить девушке на 22-летие?,


## Symmary

In [13]:
!pip3 install sumy
import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 97 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 10.1 MB 6.0 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21682 sha256=da401d634cc8cdbe08832c6ced3aa7f68672ab1174ae5370210cc47694545a72
  Stored in directory: /Users/Varvara/Library/Caches/pip/wheels/ba/9f/70/7795228568b81b57a8932755938da9fb1f291b0576752604aa
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681832 sha256=b607295e13bc531d7f71ae51ce1e92f56e957c163d1bf74bc2d55b4722c6cfad
  Stored in directory: /Users/Varvara/Library/Caches/pip/wheels/47/15/92/e6dc85fcb0686c82e1edbcfdf80cfe4808c058813fed0baa8f
Successfully built breadability pycountry
  Consider adding this directory to PAT

In [24]:
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer

In [93]:
def make_context(text):
    LANGUAGE = 'russian'
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)

    summarizer = Summarizer(stemmer)
    summary = ' '.join([str(sentence) for sentence in summarizer(parser.document, 3)])
    return summary 

In [103]:
# Чтобы избежать ручного вычленения сути ответа из большого ответа было принято использовать symmary контекста
# sumy – вычленяет заданное количество главных предложений из пассажа текста.
for ind in df_love.index:
    df_love['answer'][ind] = make_context(df_love['context'][ind])

/Users/Varvara/Library/Python/3.9/lib/python/site-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (1) is lower than number of sentences (2). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))
/Users/Varvara/Library/Python/3.9/lib/python/site-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (2) is lower than number of sentences (4). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


In [104]:
df_love

,context,question,answer
0,Она меня бьёт если не включаю Нет Нет)) Нет- с...,Девушки вопрос немного эротичный... смотрите л...,реала вполне хватает)) смотрят.. но отмазывают...
1,"верю.. Верю, но сдается мне живет он, где то н...","Вы хоть не верите в сказку, что каждого ждет г...","Тем более, что личный солидный опыт доказывает..."
2,"Куда? главное, шо не меня, так что да. это лес...",Я видел на остановке девушка целует другую дев...,"главное, шо не меня, так что да. это лесби нор..."
3,на Прочие взаимоотношения расскажи ) ❤❤❤❤❤❤❤❤❤...,девчат вы хоть понимаете на что подписываетесь?,на Прочие взаимоотношения расскажи ) ❤❤❤❤❤❤❤❤❤...
4,"Дружбы не существует, есть лишь взаимовыгодные...",бывает ли дружба между маленькой 15 летней дев...,"Дружбы не существует, есть лишь взаимовыгодные..."
...,...,...,...
57833,все что хочешь) только не указывай инициалы и ...,что написать на листочке девушке в школе? чтоб...,все что хочешь) только не указывай инициалы и ...
57834,скульптура мужик какой то Если сказали те тп=ч...,Вы видели Афродиту?))) скульптуру.. ну как кра...,скульптура мужик какой то Если сказали те тп=ч...
57835,Отзывчивый альтруист Самый верный друг Друг Ал...,"Как назвать человека, который помогает, даде е...",Отзывчивый альтруист Самый верный друг Друг Ал...
57836,"Себя Стриптизёра. Парня. Незабываемый вечер, н...",Какой подарок подарить девушке на 22-летие?,отдай наличкой маловато для подарка Я уверена ...


## Психология 

In [109]:
# скрываем имена
def preprocess(text):
    doc = nlp(text)
    names = [ent for ent in doc.ents if ent.label_ == 'PER' and ')' not in str(ent)]
    if len(names) != 0:
        for n in names:
            # избегаем попадания местоимения в имена
            if morph.parse(str(n))[0].normal_form != 'ваш' and morph.parse(str(n))[0].normal_form != 'вы':
                text = re.sub(' +', ' ', text)
                text = text.replace(str(n), '{{NAME}}')
    return text


In [110]:
def get_psy_data(filename):
    psy_data = []
    with open(filename, encoding='utf-8') as file:
        reader = file.read()
        for obj in reader.split('PROMPT:')[1:]:
            inf = {}
            for part in obj.split('\n'):
                if part.startswith(' спрашивает:'):
                    resp = re.search(r'\)(.*)', obj)
                    if resp != None:
                        q = " ".join(nltk.sent_tokenize(resp.group(1))[:3])
                        inf['question'] = preprocess(q)

                elif part.startswith('<s>'):
                    # избавляемся от следов парсинга и скрываем персональную информацию
                    inf['context'] = scrubber.clean(unicodedata.normalize("NFKC", preprocess(part).strip('<s>')))
                    inf['answer'] = make_context(inf['context'])
            if 'question' in inf:
                psy_data.append(inf)  
    return psy_data

In [111]:
train_psy = get_psy_data('психология/train_psy.txt')
print(len(train_psy))

5647


In [112]:
test_psy = get_psy_data('психология/test_psy.txt')
print(len(test_psy))

1528


In [113]:
df_psy = pd.DataFrame.from_dict(train_psy)
df_psy.head(3)

,question,context,answer
0,Здравствуйте.Меня зовут {{NAME}}.Помогите пожа...,Здавствуйте! Может быть этот вопрос обсудить с...,Может быть этот вопрос обсудить с женой? Супру...
1,Здравствуйте! Для чего психолог предложил доч...,"Здравствуйте, {{NAME}}. Вас легко понять. Если...","Если ваша дочь уже взрослый человек, то есть с..."
2,Я рассталась со своим молодым человеком три ме...,Время лечит. Нужно только принять ответственно...,"То, что ваша интуиция предупреждала Вас еще за..."


In [115]:
# пример
for ind in df_psy.index[7:8]:
    print(df_psy['question'][ind])
    print('')
    print(df_psy['context'][ind])
    print('')
    print(df_psy['answer'][ind])

Хочу получить совет и помощь. В ситуациях когда я волнуюсь, у меня начали сильно трястись руки. Раньше такого за собой не замечал.

Очевидно, что у Вас закрепилось неправильное эмоциональное реагирования на данный тип ситуаций – общение с начальством. Это негативная автоматическая привычка,  закрепившееся в результате травмирующего переживания. Подобным образом, у нас может и повышаться настроение, когда мы видим на календаре приближение Нового Года (по крайней мене у многих из нас...). Или, если мы вспомни о хорошо проведенном отдыхе на море, у нас может не только подняться настроение, но даже дыхание углубиться, и, если присутствовала, пройти головная боль.   Работать с вашей проблемой можно и нужно.  Можете перед общением с вашим директором на пару секунд закрывать глаза, и вспоминать ситуацию, в которой вы были уверены и спокойны как удав, можете представлять своего начальника уменьшенным в несколько раз. Поэкспериментируйте с этим – это может быть увлекательным. Если самостоятельн

In [116]:
# меняем порядок для дальнейшего обьединения
cols = ['context', 'question', 'answer']
df_psy = df_psy[cols]
df_psy

,context,question,answer
0,Здавствуйте! Может быть этот вопрос обсудить с...,Здравствуйте.Меня зовут {{NAME}}.Помогите пожа...,Может быть этот вопрос обсудить с женой? Супру...
1,"Здравствуйте, {{NAME}}. Вас легко понять. Если...",Здравствуйте! Для чего психолог предложил доч...,"Если ваша дочь уже взрослый человек, то есть с..."
2,Время лечит. Нужно только принять ответственно...,Я рассталась со своим молодым человеком три ме...,"То, что ваша интуиция предупреждала Вас еще за..."
3,Добрый вечер {{NAME}}! Агрессия это естественн...,Здравствуйте {{NAME}}. Помогите пожалуйста у м...,"Вы отмечаете связь вашей агрессии с обидой, не..."
4,"Лера, зачем Вы делаете вид, что Вас не задев...",Мне очень нужен совет. Мой муж меня постоянно ...,"Не копите в себе обиды, постарайтесь научиться..."
...,...,...,...
5642,"Здравствуйте,{{NAME}}.В основе Вашего дискомфо...",Здравствуйте. Немногим больше 3-х месяцев наза...,"Здравствуйте,{{NAME}}.В основе Вашего дискомфо..."
5643,"Нужна любимая, добрая, понимающая мама и огром...",Моя дочь ни чего не есть уже неделю падает в ...,"Жду вашего письма {{NAME}}, здравствуйте.Когда..."
5644,"Здравствуйте, {{NAME}}! Да, такое встречается ...",Здравствуйте. Меня бросил гражданский муж. Мы ...,"Вы осознали, что начали обесценивать отношения..."
5645,"Здравствуйте,{{NAME}}.Я думаю,вопрос звучит го...",Очень многие какие-то личные переживания я все...,А для этого им нужно об этом сказать.Обращатьс...


In [139]:
final_test_dataset = pd.concat([df_love, df_psy], ignore_index= True )
final_test_dataset.to_csv('final2_train_dataset.csv')

In [138]:
final_test_dataset

,context,question,answer
0,Она меня бьёт если не включаю Нет Нет)) Нет- с...,Девушки вопрос немного эротичный... смотрите л...,Девушки вопрос немного эротичный... смотрите л...
1,"верю.. Верю, но сдается мне живет он, где то н...","Вы хоть не верите в сказку, что каждого ждет г...","Тем более, что личный солидный опыт доказывает..."
2,"Куда? главное, шо не меня, так что да. это лес...",Я видел на остановке девушка целует другую дев...,Я видел на остановке девушка целует другую дев...
3,на Прочие взаимоотношения расскажи ) ❤❤❤❤❤❤❤❤❤...,девчат вы хоть понимаете на что подписываетесь?,девчат вы хоть понимаете на что подписываетесь...
4,"Дружбы не существует, есть лишь взаимовыгодные...",бывает ли дружба между маленькой 15 летней дев...,бывает ли дружба между маленькой 15 летней дев...
...,...,...,...
63480,"Здравствуйте,{{NAME}}.В основе Вашего дискомфо...",Здравствуйте. Немногим больше 3-х месяцев наза...,Здравствуйте. Немногим больше 3-х месяцев наза...
63481,"Нужна любимая, добрая, понимающая мама и огром...",Моя дочь ни чего не есть уже неделю падает в ...,Моя дочь ни чего не есть уже неделю падает в ...
63482,"Здравствуйте, {{NAME}}! Да, такое встречается ...",Здравствуйте. Меня бросил гражданский муж. Мы ...,Здравствуйте. Меня бросил гражданский муж. Мы ...
63483,"Здравствуйте,{{NAME}}.Я думаю,вопрос звучит го...",Очень многие какие-то личные переживания я все...,Очень многие какие-то личные переживания я все...


In [133]:
# для GPT-3 для генерации нужно было чуть переделать датасет:
# склеивали мы в последнем датасете потому что GPT-3
# просто продолжает текст из какой-то "затравки"
# и соответственно вся нужная инфа должна содержаться в ней.
# То есть когда она обучится мы будем давать такой вход модели.
# PROMPT: тут наш вопрос | и тут наш контекст
# ANSWER:

for ind in final_test_dataset.index:
    final_test_dataset['answer'][ind] = final_test_dataset['question'][ind] + final_test_dataset['context'][ind]

In [137]:
final_test_dataset.to_csv('final3_train_dataset.csv')